# 1. Importação das bilbiotecas necessárias

In [1]:
# Importando o pandas
import pandas as pd

# Importando biblioteca para trabalhar com o MongoDB
import pymongo as pym

# Importando o numpy para trabalhar com cálculos matemáticos
import numpy as np

#Importando o seaborn e o matplotlib para análises gráficas
import seaborn as sns
import matplotlib.pyplot as plt

# 2. Carga do arquivo e dados iniciais

In [2]:
# Como o arquivo dos locais é maior, a seleção das colunas utilizadas será feita  antes da importação do csv

colunas = ['CO_IES','CO_LOCAL_OFERTA','NO_LOCAL_OFERTA','CO_CURSO_POLO','CO_CURSO','CO_UF',
          'CO_MUNICIPIO','IN_CAPITAL','IN_SEDE']

In [3]:
# Lendo o arquivo CSV disponível nos microdados do Censo da Educação Superior e gravando
# ele em um DataFrame
locais = pd.read_csv('../data/SUP_LOCAL_OFERTA_2019.CSV', sep="|", encoding="latin-1", usecols=colunas)

In [4]:
locais.shape

(1254044, 9)

In [5]:
# Verificar a quantidade de memória ocupada 
locais.info(memory_usage="deep")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1254044 entries, 0 to 1254043
Data columns (total 9 columns):
 #   Column           Non-Null Count    Dtype  
---  ------           --------------    -----  
 0   CO_LOCAL_OFERTA  1254044 non-null  int64  
 1   NO_LOCAL_OFERTA  1254044 non-null  object 
 2   CO_IES           1254044 non-null  int64  
 3   CO_UF            1251118 non-null  float64
 4   CO_MUNICIPIO     1254044 non-null  int64  
 5   IN_CAPITAL       1251118 non-null  float64
 6   IN_SEDE          637919 non-null   float64
 7   CO_CURSO_POLO    1217866 non-null  float64
 8   CO_CURSO         1254044 non-null  int64  
dtypes: float64(4), int64(4), object(1)
memory usage: 177.3 MB


In [6]:
locais.dtypes

CO_LOCAL_OFERTA      int64
NO_LOCAL_OFERTA     object
CO_IES               int64
CO_UF              float64
CO_MUNICIPIO         int64
IN_CAPITAL         float64
IN_SEDE            float64
CO_CURSO_POLO      float64
CO_CURSO             int64
dtype: object

In [7]:
locais.head()

,CO_LOCAL_OFERTA,NO_LOCAL_OFERTA,CO_IES,CO_UF,CO_MUNICIPIO,IN_CAPITAL,IN_SEDE,CO_CURSO_POLO,CO_CURSO
0,105340,Campus Universitário de Cuiabá,1,51.0,5103403,1.0,1.0,NaN,1
1,105340,Campus Universitário de Cuiabá,1,51.0,5103403,1.0,1.0,NaN,2
2,105340,Campus Universitário de Cuiabá,1,51.0,5103403,1.0,1.0,NaN,3
3,105340,Campus Universitário de Cuiabá,1,51.0,5103403,1.0,1.0,NaN,6
4,105340,Campus Universitário de Cuiabá,1,51.0,5103403,1.0,1.0,NaN,7


In [8]:
# Importando arquivo de cidades do IBGE que servirá para popular o endereço dos locais
colunas = ['codigo_ibge','NO_MUNICIPIO','LAT_MUNICIPIO','LON_MUNICIPIO','CO_UF','SG_UF','NO_UF']
municipios = pd.read_csv('../data/municipios.csv', sep=",", header=0, names=colunas)

In [9]:
municipios.shape

(5570, 7)

In [10]:
municipios.head(3)

,codigo_ibge,NO_MUNICIPIO,LAT_MUNICIPIO,LON_MUNICIPIO,CO_UF,SG_UF,NO_UF
0,5200050,Abadia de Goiás,-16.7573,-49.4412,52,GO,Goiás
1,3100104,Abadia dos Dourados,-18.4831,-47.3916,31,MG,Minas Gerais
2,5200100,Abadiânia,-16197.0000,-48.7057,52,GO,Goiás


In [11]:
municipios.dtypes

codigo_ibge        int64
NO_MUNICIPIO      object
LAT_MUNICIPIO    float64
LON_MUNICIPIO    float64
CO_UF              int64
SG_UF             object
NO_UF             object
dtype: object

# 3. Tratando os dados do dataset

In [12]:
# Removendo colunas desnecessárias
municipios.drop(['CO_UF'], axis=1, inplace = True)

In [13]:
# Criando um DataFrame somente com as colunas de endereço das IES 
# E outro com as colunas consideradas na estrutura do trabalho
endereco_locais = locais[{'CO_UF','CO_MUNICIPIO','IN_CAPITAL'}]
locais = locais [{'CO_LOCAL_OFERTA','NO_LOCAL_OFERTA','CO_IES','IN_SEDE','CO_CURSO_POLO','CO_CURSO'}]

In [14]:
# Adicionando uma coluna Endereco no DataFrame dos locais, pois por padrão
# essa coluna não existe
locais['ENDERECO'] = ""

In [15]:
## Categorizando o IN_CAPITAL do endereco 
# 1) Alterando o tipo dos dados int, para object (string)
endereco_locais['IN_CAPITAL'] = endereco_locais['IN_CAPITAL'].apply(str)

In [16]:
# Alterando os valores por suas respectivas categorias  
# 2) IN CAPITAL 0: Não; 1: Sim. 
endereco_locais.loc[endereco_locais['IN_CAPITAL'] == '0', 'IN_CAPITAL'] = 'Não'
endereco_locais.loc[endereco_locais['IN_CAPITAL'] == '1', 'IN_CAPITAL'] = 'Sim'

In [17]:
# fazendo um merge de endereço com municipios
endereco_locais = endereco_locais.merge(municipios, how='left', left_on='CO_MUNICIPIO', right_on='codigo_ibge')

In [18]:
# Criando um dicionário do Python a partir do DataFrame de enderecos
endereco_locais_dict = endereco_locais.to_dict('records')

In [19]:
# Adicionando um endereco (através do dicionário criado no passo anterior)
# e inserindo na coluna "ENDERECO" de cada LOCAL
for i in locais.index:
    locais.at[i,'ENDERECO'] = endereco_locais_dict[i]  

In [20]:
## Verificando se existem informações nulas 
locais.isnull().sum()

CO_IES                  0
CO_CURSO_POLO       36178
CO_LOCAL_OFERTA         0
IN_SEDE            616125
CO_CURSO                0
NO_LOCAL_OFERTA         0
ENDERECO                0
dtype: int64

In [21]:
## Alterando os valores nulos do atributo "IN_SEDE" para 2.0 
locais['IN_SEDE'].fillna(2.0, inplace = True)

In [22]:
# 1) Alterando o tipo dos dados int e float, para object (string), afim de categorizar 
locais['IN_SEDE'] = locais['IN_SEDE'].apply(str)

In [23]:
locais.head()

,CO_IES,CO_CURSO_POLO,CO_LOCAL_OFERTA,IN_SEDE,CO_CURSO,NO_LOCAL_OFERTA,ENDERECO
0,1,NaN,105340,1.0,1,Campus Universitário de Cuiabá,"{'CO_UF': 51.0, 'CO_MUNICIPIO': 5103403, 'IN_C..."
1,1,NaN,105340,1.0,2,Campus Universitário de Cuiabá,"{'CO_UF': 51.0, 'CO_MUNICIPIO': 5103403, 'IN_C..."
2,1,NaN,105340,1.0,3,Campus Universitário de Cuiabá,"{'CO_UF': 51.0, 'CO_MUNICIPIO': 5103403, 'IN_C..."
3,1,NaN,105340,1.0,6,Campus Universitário de Cuiabá,"{'CO_UF': 51.0, 'CO_MUNICIPIO': 5103403, 'IN_C..."
4,1,NaN,105340,1.0,7,Campus Universitário de Cuiabá,"{'CO_UF': 51.0, 'CO_MUNICIPIO': 5103403, 'IN_C..."


In [24]:
# Alterando os valores por suas respectivas categorias  
# 2) IN_SEDE: 0.0: Não; 1.0: Sim; 2.0: "Não informado"  
locais.loc[locais['IN_SEDE'] == '0.0', 'IN_SEDE'] = 'Não'
locais.loc[locais['IN_SEDE'] == '1.0', 'IN_SEDE'] = 'Sim'
locais.loc[locais['IN_SEDE'] == '2.0', 'IN_SEDE'] = 'Não informado'

## 4. Análise dos dados dos locais de oferta


In [25]:
print("Quantidade de linhas: ", locais.shape[0])
print("Quantidade de colunas: ",locais.shape[1])

Quantidade de linhas:  1254044
Quantidade de colunas:  7


In [26]:
locais.describe()

,CO_IES,CO_CURSO_POLO,CO_LOCAL_OFERTA,CO_CURSO
count,1.254044e+06,1.217866e+06,1.254044e+06,1.254044e+06
mean,1.257400e+03,8.980439e+05,1.001589e+06,1.207591e+06
std,2.179965e+03,4.660015e+05,5.330635e+05,5.389890e+05
min,1.000000e+00,1.000000e+00,1.930000e+02,1.000000e+00
25%,3.220000e+02,5.109602e+05,1.060403e+06,1.204869e+06
50%,6.710000e+02,9.107335e+05,1.089915e+06,1.377504e+06
75%,1.491000e+03,1.281515e+06,1.102763e+06,1.443692e+06
max,2.467200e+04,1.711742e+06,2.500011e+06,5.001534e+06


# 5. Envio dos dados para o mongoDB

In [27]:
# Criando a conexão com o mongoDB
client = pym.MongoClient('mongodb://localhost:27017/')

# Acessando a base de dados criada
db = client['censo']

# Acessando a coleção locais de oferta 
collection_locais =  db['locais_de_oferta']

In [28]:
# Transformando o DataSet dos locais em um dicionario com o objetivo
# de salvar ele no MongoDB, pois o banco não aceita estruturas do tipo DataFrame
locais_dict = locais.to_dict('records')

In [29]:
# Enviando os dados para o MongoDB
collection_locais.insert_many(locais_dict)